In [15]:
import calendar
from datetime import datetime, timedelta
import time  
import pandas as pd
import QuantLib as ql
import os

# *skip*

In [6]:
def get_week_of_month(year, month, day):
    # 周一为一周的开始
    begin = int(datetime.date(year, month, 1).strftime("%W"))
    end = int(datetime.date(year, month, day).strftime("%W"))

    return end - begin + 1


print(get_week_of_month(2021, 1, 31))
print(get_week_of_month(2021, 2, 28))
print(get_week_of_month(2021, 2, 21))


5
4
3


In [11]:
def get_third_Friday(year, month):
    first_day_of_month = datetime.date(year, month, 1)
    first_friday = first_day_of_month + timedelta(days=((4-calendar.monthrange(year,month)[0])+7)%7)
    
    return first_friday + timedelta(days=14)

print(get_third_Friday(2021, 1))
print(get_third_Friday(2021, 2))
print(get_third_Friday(2021, 3))
'''
now = datetime.now()
first_day_of_month = datetime(now.year, now.month, 1)
first_friday = first_day_of_month + timedelta(days=((4-calendar.monthrange(now.year,now.month)[0])+7)%7)
# 4 is friday of week
third_friday = first_friday + timedelta(days=14)
'''

# http://hk.uwenku.com/question/p-ykrhejto-qv.html

2021-01-15
2021-02-19
2021-03-19


'\nnow = datetime.now()\nfirst_day_of_month = datetime(now.year, now.month, 1)\nfirst_friday = first_day_of_month + timedelta(days=((4-calendar.monthrange(now.year,now.month)[0])+7)%7)\n# 4 is friday of week\nthird_friday = first_friday + timedelta(days=14)\n'

In [ ]:
us_calendar = ql.UnitedStates()

close = []
for d in date_list['before 11']:
    if us_calendar.isHoliday(ql.Date(d.day, d.month, d.year)):
        close.append(d)

# 確認前一日是否仍為假日，若為假日則再前一天
before_1D = []
for d in close:
    d_before = d - timedelta(days = 1)
    while(us_calendar.isHoliday(ql.Date(d_before.day, d_before.month, d_before.year))):
        d_before = d_before - timedelta(days = 1)
    before_1D.append(d_before)

pd.DataFrame(before_1D)


close_2 = []
for d2 in before_1D:
    if us_calendar.isHoliday(ql.Date(d2.day, d2.month, d2.year)):
        close_2.append(d2)
pd.DataFrame(close_2)


weekday = []
for d2 in before_1D:
    weekday.append(datetime(d2.year, d2.month, d2.day).strftime("%w"))  # 星期，星期日為0

pd.DataFrame(weekday)


In [ ]:
# https://www.cnpython.com/qa/52368
import holidays

# Select country
us_holidays = holidays.US()

# Print all the holidays in US in year 2018
for ptr in holidays.US(years = 2007).items():
    print(ptr)

# 發布訊號日期，最後交易日前11、21天的日期

In [14]:
def option_expiration(year, month): 
    day = 21 - (calendar.weekday(year, month, 1) + 2) % 7 
    return datetime.date(datetime(year, month, day))

def is_holiday(date):
    # 確認前一日是否仍為假日，若為假日則再前一天
    df = pd.read_excel('D:\\碩論資料\\耶穌受難日.xlsx')['good Friday'].to_list()
    good_fri = [str(datetime.date(i)) for i in df]
    while(us_calendar.isHoliday(ql.Date(date.day, date.month, date.year)) or str(date) in good_fri):
        date = date - timedelta(days = 1)
    return date


# 取得最後交易日前11、21天的日期
us_calendar = ql.UnitedStates()

date_list = []
for y in range(1996, 2018):
    for m in range(1,13):
        day = is_holiday(option_expiration(y, m))
        day_11 = is_holiday(day - timedelta(days = 11))
        day_21 = is_holiday(day - timedelta(days = 21))
        date_list.append((day, day_11, day_21))

'''
# 為 'before 21' 多提供一天資訊 ?
day = option_expiration(2018, 1)
day_11 = day - timedelta(days = 11)
day_21 = day - timedelta(days = 21)
date_list.append((day, day_11, day_21))
'''

date_list = pd.DataFrame(date_list, columns = ['expiration date', 'before 11', 'before 21'])
date_list = date_list.drop(index = [0])
date_list

,expiration date,before 11,before 21
1,1996-02-16,1996-02-05,1996-01-26
2,1996-03-15,1996-03-04,1996-02-23
3,1996-04-19,1996-04-08,1996-03-29
4,1996-05-17,1996-05-06,1996-04-26
5,1996-06-21,1996-06-10,1996-05-31
...,...,...,...
259,2017-08-18,2017-08-07,2017-07-28
260,2017-09-15,2017-09-01,2017-08-25
261,2017-10-20,2017-10-06,2017-09-29
262,2017-11-17,2017-11-06,2017-10-27


# 新增變數vol_2

In [ ]:
dir_name = 'C:\\Users\\User\\Desktop\\資料整理\\semi_55_option_0'
files = os.listdir(dir_name)
for file in files:
    #print(file)
    file_path = os.path.join(dir_name, file)
    f = pd.read_csv(file_path)
    f['weeklyvol_2'] = f['weeklyputvol'] / (f['weeklycallvol']+f['weeklystockvol'])
    #f.to_csv('C:\\Users\\User\\Desktop\\資料整理\\semi_55_option\\' + file[0:file.find('.')] + '.csv', index=False)


# skew, vol 算平均

In [17]:
df_1 = pd.DataFrame()  # skew
df_2 = pd.DataFrame()  # vol
df_3 = pd.DataFrame()  # vol_2

col_list = ['date']
date_str = date_list.copy()
date_str = date_str.astype(str)

dir_name = 'C:\\Users\\User\\Desktop\\資料整理\\semi_55_option'
files = os.listdir(dir_name)
for file in files:       
    file_path = os.path.join(dir_name, file)
    f = pd.read_csv(file_path)
    col_list.append(file[0:file.find('_')])  # 公司名稱

    if len(col_list) == 2:
        df_1 = f[f['date'].isin([str(i) for i in date_list['before 11'].to_list()])][['date', 'weeklyskew']]
        df_2 = f[f['date'].isin([str(i) for i in date_list['before 11'].to_list()])][['date', 'weeklyvol']]
        df_3 = f[f['date'].isin([str(i) for i in date_list['before 11'].to_list()])][['date', 'weeklyvol_2']]
    else:
        temp_1 = f[f['date'].isin([str(i) for i in date_list['before 11'].to_list()])][['date', 'weeklyskew']]
        temp_2 = f[f['date'].isin([str(i) for i in date_list['before 11'].to_list()])][['date', 'weeklyvol']]
        temp_3 = f[f['date'].isin([str(i) for i in date_list['before 11'].to_list()])][['date', 'weeklyvol_2']]
        df_1 = pd.merge(df_1, temp_1, on = 'date', how = 'outer')
        df_2 = pd.merge(df_2, temp_2, on = 'date', how = 'outer')
        df_3 = pd.merge(df_3, temp_3, on = 'date', how = 'outer')

df_1.columns = col_list
df_2.columns = col_list
df_3.columns = col_list
df_1.sort_values(by = ['date'], ascending = True, inplace = True)
df_2.sort_values(by = ['date'], ascending = True, inplace = True)
df_3.sort_values(by = ['date'], ascending = True, inplace = True)
name_1 = 'Total weeklyskew_b11'
name_2 = 'Total weeklyvol_b11'
name_3 = 'Total weeklyvol_2_b11'
#df_1.to_csv('C:\\Users\\User\\Desktop\\資料整理\\semi_skew&vol\\' + name_1 + '.csv', index=False)
#df_2.to_csv('C:\\Users\\User\\Desktop\\資料整理\\semi_skew&vol\\' + name_2 + '.csv', index=False)
#df_3.to_csv('C:\\Users\\User\\Desktop\\資料整理\\semi_skew&vol\\' + name_3 + '.csv', index=False)


# 計算平均，無該月資料的公司在計算平均值不算進母體數
semi_mean = pd.DataFrame(columns = ['skew', 'vol', 'vol_2'], index = df_1['date'])
for i in range(len(df_1)):
    semi_mean.iloc[i][0] = df_1.mean(axis=1).to_frame().iloc[i][0]
    semi_mean.iloc[i][1] = df_2.mean(axis=1).to_frame().iloc[i][0]
    semi_mean.iloc[i][2] = df_3.mean(axis=1).to_frame().iloc[i][0]

name = 'mean_b11'
#semi_mean.to_csv('C:\\Users\\User\\Desktop\\資料整理\\semi_skew&vol\\' + name + '.csv', index=True)

In [18]:
semi_mean

,skew,vol,vol_2
date,,,
1996-02-05,0.0447298,0.220429,0.316522
1996-03-04,0.0309609,0.311313,0.567502
1996-04-08,0.0375525,0.268198,0.442574
1996-05-06,0.0355166,0.257,0.391721
1996-06-10,0.0191947,0.289212,0.457521
...,...,...,...
2017-08-07,0.0324078,0.304394,0.520162
2017-09-01,0.0303249,0.267063,0.423728
2017-10-06,0.0289013,0.248124,0.413673
